# Examples for Probing LMs with RICA

## Loading RICA Test data

In [104]:
test_sentences=[]
with open('data/joint_test_set/test_sentences.txt', 'r') as f:
    for line in f:
        test_sentences.append(line.replace('\n',''))

test_sentences_masks = []
with open('data/joint_test_set/test_sentences_m.txt', 'r') as f:
    for line in f:
        test_sentences_masks.append(line.replace('\n',''))
      
import ast
test_sentences_mask_pairs = []
with open('data/joint_test_set/config.txt', 'r') as f:
    for line in f:
        test_sentences_mask_pairs.append(ast.literal_eval(line))

## Convert to masked statement format for probing

In [35]:
masked_test_sentences = []

counter=0
for test_sentence in test_sentences:
    masked_word = test_sentences_masks[counter]
    k = test_sentence.rfind(masked_word)
    new_string = test_sentence[:k] + "[MASK]" + test_sentence[k+len(masked_word):]
    masked_test_sentences.append(new_string)
    counter+=1

In [38]:
masked_test_sentences[:5]

['fhvpltjpwcd is made out of paper and kuna is made out of metal, so fhvpltjpwcd is [MASK] likely to crumple than kuna',
 'kuna is made of paper, fhvpltjpwcd is made of metal, so fhvpltjpwcd is [MASK] likely to crumple than kuna',
 'fhvpltjpwcd is made of paper, kuna is made of metal, so kuna is [MASK] likely to crumple than fhvpltjpwcd',
 'lnbnzthevf is made out of paper and nzwrmmw is made out of metal, so lnbnzthevf is not [MASK] likely to crumple than nzwrmmw',
 'nzwrmmw is made out of paper and lnbnzthevf is made out of metal, so lnbnzthevf is not [MASK] likely to crumple than nzwrmmw']

## Loading model from latest Huggingface pipeline for masked word prediction
### For fine-tuned models, check our instructions on README and replace model with your model file address

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased',top_k=100)
# For fine-gtu
unmasker(masked_test_sentences[53])


In [ ]:
bert_base_predictions = []

counter=0
for masked_statement in masked_test_sentences:
    bert_base_predictions.append(unmasker(masked_statement))
    counter+=1
    print(counter)

In [105]:
bert_base_masked_word_predictions = []
correct = 0 

counter=0
for lst in bert_base_predictions:
    correct_word = test_sentences_masks[counter]
    test_sentences_mask_pairs[counter].remove(correct_word)
    incorrect_word = test_sentences_mask_pairs[counter][0]
    
    correct_shown = False
    incorrect_shown = False
    for dic in lst:
        token = dic['token_str']
        if token == correct_word and incorrect_shown == False:
            bert_base_masked_word_predictions.append(correct_word)
            correct_shown=True
            correct+=1
            continue
        elif token == incorrect_word and correct_shown == False:
            bert_base_masked_word_predictions.append(incorrect_word)
            incorrect_shown=True
            continue
    if correct_shown == False and incorrect_shown == False:
        bert_base_masked_word_predictions.append(incorrect_word)
    counter+=1

In [106]:
correct/len(bert_base_masked_word_predictions)

0.4925143953934741